## 복수선택형 문제

In [1]:
from tools.core.llm_query import LLMQuery

llm = LLMQuery()
system_prompt = """당신은 프롬프트 엔지니어입니다. 아래 예시처럼 '새로운 질문'을 만드는 프롬프트를 짜려고 합니다. 

1. question과 original_question을 비교하고, 차이를 확인하세요. 
2. 옳은 것을 고르는 문제인지, 옳지 않은 것을 고르는 문제인지는 question에 따릅니다. 
3. question이 '어떤 주제에 대해' 옳은 것/옳지 않은 것을 고르는 것인지 명확하지 않다면 original_question의 질문과 questions의 '옳은 것/옳지 않은 것'을 조합해 질문을 만드세요. 
4. question에 주제가 있고, 옳은 것/옳지 않은 것을 고르는 것이 명확하다면 question을 그대로 사용하세요.

예시
'question': '옳은 것을 모두 고르시오.'
'original_question': '고객과 생산적인 대화를 위한 방법으로 다음 설명 중 가장 적절하지 않은 것은?'
-> 새로운 질문: '고객과 생산적인 대화를 위한 방법으로 다음 설명 중 옳은 것을 모두 고르시오.'
"""

# target_answer_count = 3
# options_ct = 5
# question = {}
# question['answer'] = "②"


user_prompt = f""""""
response = llm.query_openrouter(system_prompt, user_prompt)

In [2]:
print(response)

아래 프롬프트를 그대로 사용하세요.

작업:
- 주어진 question과 original_question을 비교해 ‘새로운 질문’ 한 줄을 생성한다.

입력:
- question: {여기에 입력}
- original_question: {여기에 입력}

규칙:
1) question과 original_question을 비교해
   - 주제(무엇에 대한 질문인지)
   - 판단 방향(옳은 것 vs. 옳지 않은 것, 적절한 vs. 부적절한 등)을 파악한다.
2) 최종 판단 방향은 question을 따른다. question에 방향이 없으면 original_question을 따른다.
3) question에 주제가 없거나 불명확하면, original_question에서 주제만 추출해 question의 형식/방향과 결합한다.
   - 주제 추출 시 ‘다음 중/설명 중/가장/옳은/옳지 않은/적절(하지) 않은/맞는/틀린/해당하지 않는’ 등 판단 표현은 제거하고, 대상/맥락만 남긴다.
4) question에 주제와 판단 방향이 모두 명확하면 question을 그대로 사용한다.
5) 어투와 종결(‘…고르시오.’, ‘…것은?’ 등)은 question의 형식을 최대한 유지한다.
6) 의미 왜곡 없이 자연스러운 한국어로 다듬되, 불필요한 수식어는 추가하지 않는다.
7) 출력 형식은 한 줄로만: 새로운 질문: {문장}

예시 입력:
- question: 옳은 것을 모두 고르시오.
- original_question: 고객과 생산적인 대화를 위한 방법으로 다음 설명 중 가장 적절하지 않은 것은?

예시 출력:
새로운 질문: 고객과 생산적인 대화를 위한 방법으로 다음 설명 중 옳은 것을 모두 고르시오.


In [ ]:
system_prompt = f"""당신은 25년 경력의 문제 출제 전문가입니다.

검증
- 목표 정답 수(= '옳지 않은 것' 개수, {target_answer_count})가 2 이상 {options_ct} 이하인지 확인합니다. 범위를 벗어나면 오류를 보고합니다.
- 선택지 수({options_ct})와 순서는 반드시 유지합니다.

변형 규칙
- 문제 지문은 “옳지 않은 것을 모두 고르시오”로 바꿉니다. 그 외 본문, 수식(LaTeX), 표, 선택지 문구(변형 대상 제외)는 원형을 최대한 보존합니다.
- 목표 오답 수 = '옳은 것' 개수 = {options_ct - target_answer_count}.
  - 목표 오답 수 = 0: 원래 오답({question.get('answer', '')})을 최소 수정으로 ‘옳지 않음’으로 뒤집습니다. 그 외 선택지는 변경하지 않습니다. 결과적으로 모든 선택지가 옳지 않음입니다.
  - 목표 오답 수 = 1: 원래 오답을 그대로 유지합니다. 그 외 선택지는 변경하지 않습니다. 결과적으로 원래 오답인 '옳은 것' 1개만 남습니다(단일선택형 재검증).
  - 목표 오답 수 ≥ 2: 원래 오답을 그대로 유지하고, 추가로 (목표 오답 수 - 1)개 만큼 원래 옳지 않았던 선택지를 골라 최소 수정으로 ‘옳음’으로 만듭니다.
- 최소 수정 원칙
  - 허용되는 변경 예: 수치/단위/부등호/최대↔최소/있다↔없다/반드시↔경우도 있다/조건의 범위·한정어 조정 등.
  - 제공된 해설의 논리 범위를 벗어나는 임의 창작 금지. 외부 사실 의존 금지.
  - 변형 대상이 아닌 선택지의 문구는 절대 수정하지 않습니다.
- 선택지 선정 가이드
  - 변형이 필요한 경우, 가장 적은 토큰 변경으로 참⇄거짓을 뒤집기 쉬운 선택지부터 우선 선택합니다.
  - 의미 일관성 유지: 변형으로 인해 다른 선택지와 모순되거나 문제 전체의 전제가 깨지지 않도록 합니다.

정답과 설명
- 정답(answer)은 변형 후 ‘옳지 않은’ 선택지의 번호 목록입니다. 번호는 "①","②",... 형식의 문자열로, 오름차순으로 정렬합니다.
- explanation에는 모든 선택지를 순회하며 다음 형식으로 간단·명확히 기술합니다.
  - “① 옳다(원래 오답): 근거 …”
  - “③ 옳지 않다: 근거 …”
  - “⑤ 옳다(변형): 변경 ‘높다→낮다’, 근거 …”
- 변형된 선택지는 반드시 어떤 단어/수치/기호를 어떻게 바꿨는지 구체적으로 한 토큰 수준으로 표기합니다.

출력 형식(JSON)
{{
  "question_id": "문제번호",
  "question": "변형된 문제(옳지 않은 것을 모두 고르시오)",
  "options": ["① 선택지1", "② 선택지2", ...],  
  "answer": ["정답번호1", "정답번호2", ...], 
  "explanation": "① 옳다(원래 오답): … / ③ 옳지 않다: … / ⑤ 옳다(변형): 변경 ‘…→…’, 근거 …"
}}

비고
- 목표 오답 수 계산식으로 일관되게 제어합니다.
  - 추가로 만들어야 할 오답 수 = {max(0, options_ct - target_answer_count - 1)}.
  - 단, {options_ct - target_answer_count} = 0이면 원래 오답을 ‘옳지 않음’으로 뒤집고 추가 오답은 0입니다.
- 선택지 레이블(①, ②, …)과 원문 순서를 유지합니다.
- 최종적으로 옳지 않음 개수 = {target_answer_count}, 오답(옳음) 개수 = {options_ct - target_answer_count}임을 자기점검합니다.
"""

### 피드백 반영

In [ ]:
atc[132]

In [ ]:
user_prompt = f"""
=========== 문제 ==========
"""
p = atc[132]
question_id = p['file_id'] + '_' + p['tag']
question = p['question']
options = p['options']
answer = p['answer']
explanation = p['explanation']

user_prompt += f"""문제번호: {question_id}
문제: {question}
선택지: {options}
답: {answer}
해설: {explanation}
"""

In [ ]:
# 입력
# - options_ct: 원문 선택지 수
# - i: 변형 후 정답(‘옳은 것’)의 개수
# - given_wrong_idx: 원래 ‘옳지 않은’ 선택지 번호(필수)

question = atc[132]
options_ct = len(options)
target_answer_count = 2
given_wrong_idx = answer

system_prompt = f"""당신은 25년 경력의 문제 출제 전문가입니다.

검증
- 목표 정답 수(= '옳지 않은 것' 개수, {target_answer_count})가 2 이상 {options_ct} 이하인지 확인합니다. 범위를 벗어나면 오류를 보고합니다.
- 선택지 수({options_ct})와 순서는 반드시 유지합니다.

변형 규칙
- 문제 지문은 “옳지 않은 것을 모두 고르시오”로 바꿉니다. 그 외 본문, 수식(LaTeX), 표, 선택지 문구(변형 대상 제외)는 원형을 최대한 보존합니다.
- 목표 오답 수 = '옳은 것' 개수 = {options_ct - target_answer_count}.
  - 목표 오답 수 = 0: 원래 오답({question.get('answer', '')})을 최소 수정으로 ‘옳지 않음’으로 뒤집습니다. 그 외 선택지는 변경하지 않습니다. 결과적으로 모든 선택지가 옳지 않음입니다.
  - 목표 오답 수 = 1: 원래 오답을 그대로 유지합니다. 그 외 선택지는 변경하지 않습니다. 결과적으로 원래 오답인 '옳은 것' 1개만 남습니다(단일선택형 재검증).
  - 목표 오답 수 ≥ 2: 원래 오답을 그대로 유지하고, 추가로 (목표 오답 수 - 1)개 만큼 원래 옳지 않았던 선택지를 골라 최소 수정으로 ‘옳음’으로 만듭니다.
- 최소 수정 원칙
  - 허용되는 변경 예: 수치/단위/부등호/최대↔최소/있다↔없다/반드시↔경우도 있다/조건의 범위·한정어 조정 등.
  - 제공된 해설의 논리 범위를 벗어나는 임의 창작 금지. 외부 사실 의존 금지.
  - 변형 대상이 아닌 선택지의 문구는 절대 수정하지 않습니다.
- 선택지 선정 가이드
  - 변형이 필요한 경우, 가장 적은 토큰 변경으로 참⇄거짓을 뒤집기 쉬운 선택지부터 우선 선택합니다.
  - 의미 일관성 유지: 변형으로 인해 다른 선택지와 모순되거나 문제 전체의 전제가 깨지지 않도록 합니다.

정답과 설명
- 정답(answer)은 변형 후 ‘옳지 않은’ 선택지의 번호 목록입니다. 번호는 "①","②",... 형식의 문자열로, 오름차순으로 정렬합니다.
- explanation에는 모든 선택지를 순회하며 다음 형식으로 간단·명확히 기술합니다.
  - “① 옳다(원래 오답): 근거 …”
  - “③ 옳지 않다: 근거 …”
  - “⑤ 옳다(변형): 변경 ‘높다→낮다’, 근거 …”
- 변형된 선택지는 반드시 어떤 단어/수치/기호를 어떻게 바꿨는지 구체적으로 한 토큰 수준으로 표기합니다.

출력 형식(JSON)
{{
  "question_id": "문제번호",
  "question": "변형된 문제(옳지 않은 것을 모두 고르시오)",
  "options": ["① 선택지1", "② 선택지2", ...],  
  "answer": ["정답번호1", "정답번호2", ...], 
  "explanation": "① 옳다(원래 오답): … / ③ 옳지 않다: … / ⑤ 옳다(변형): 변경 ‘…→…’, 근거 …"
}}

비고
- 목표 오답 수 계산식으로 일관되게 제어합니다.
  - 추가로 만들어야 할 오답 수 = {max(0, options_ct - target_answer_count - 1)}.
  - 단, {options_ct - target_answer_count} = 0이면 원래 오답을 ‘옳지 않음’으로 뒤집고 추가 오답은 0입니다.
- 선택지 레이블(①, ②, …)과 원문 순서를 유지합니다.
- 최종적으로 옳지 않음 개수 = {target_answer_count}, 오답(옳음) 개수 = {options_ct - target_answer_count}임을 자기점검합니다.
"""

In [ ]:
from tools.core.llm_query import LLMQuery

llm = LLMQuery()
response = llm.query_openrouter(system_prompt, user_prompt, model_name = 'openai/o3')
print(response)

In [ ]:
question

## 서술형 프롬프트 짜기

In [3]:
from tools.core.llm_query import LLMQuery

llm = LLMQuery()

In [1]:
system_prompt = """당신은 프롬프트 엔지니어입니다. 아래와 같은 원리를 담아 주어진 객관식 문제를 서술형 문제로 변형하는 프롬프트를 만들려고 합니다.

1. 기존 객관식 문제의 질문에서 주제 추출 (ex. 코넥스상장법인의 기업설명회에 대한 설명 중 잘못된 것은? -> 코넥스상장법인의 기업설명회)
2. 추출한 내용을 바탕으로 서술형 문제로 변형 (ex. 코넥스상장법인의 기업설명회에 대해 서술하시오)
3. 1개의 옳지 않은 선지와 그 해설, 3~4개의 옳은 선지와 그 해설에서 각 선지별로 키워드 추출 (ex. ① 코넥스상장법인은 분기마다 기업설명회를 개최해야 한다. -> 기업설명회 개최 주기)
4. 키워드를 포함한 서술형 문제 완성 (ex. 다음 키워드를 활용하여 코넥스상장법인의 기업설명회에 관해 서술하시오.\n 키워드: 기업설명회 개최 주기, 상장폐지사유, 공시 의무, ...)

단, 키워드는 10개 이하여야 합니다. 키워드는 너무 정답이 드러나지 않도록 주의합니다.
선지와 해설만을 이용해서 서술형 문제로 변형할 수 있는 프롬프트를 작성해주세요."""

q = {'file_id': 'SS0206',
 'title': '2025 해커스 증권투자권유대행인 한권합격 핵심 개념 + 적중문제',
 'chapter': '제1과목 증권분석 및 증권시장',
 'tag': 'q_0211_0003',
 'domain': '자산운용',
 'subdomain': '투자분석 및 포트폴리오이론',
 'classification_reason': '코넥스 상장법인 기업설명회(개최 주기, 공시 의무, 신고, 상장폐지)로 증권 시장 상장 규정과 공시 중심',
 'is_calculation': False,
 'question': '코넥스상장법인의 기업설명회에 대한 설명 중 잘못된 것은?',
 'options': ['① 코넥스상장법인은 분기마다 기업설명회를 개최해야 한다.',
  '② 코넥스상장법인이 기업설명회를 개최하는 경우 일시, 장소, 설명회 내용 등을 공시해야 할 의무가 있다.',
  '③ 코넥스상장법인은 기업설명회 개최 결과를 개최일 익일까지 거래소에 신고해야 한다.',
  '④ 코넥스상장법인이 2반기 동안 연속하여 기업설명회를 개최하지 않거나, 3년 동안 4회 이상 개최하지 않은 경우 상장폐지사유에 해당한다.'],
 'answer': '①',
 'explanation': '코넥스상장법인은 반기마다 기업설명회를 개최해야 한다.',
 'answer_type': 'wrong'}

user_prompt = f"""
==== 문제 예시 ====
질문: {q['question']}
선택지: {q['options']}
정답: {q['answer']}
해설: {q['explanation']}
"""

In [4]:
response = llm.query_openrouter(system_prompt, user_prompt, model_name = 'openai/gpt-5')
print(response)

다음 지시문을 따르세요. 당신의 역할은 주어진 객관식 항목(선지)과 해설만을 근거로 문제의 핵심 주제를 추론하고, 이를 서술형 문제로 변환하는 것입니다.

입력으로 제공되는 것
- 선택지: 문자열 배열
- 정답: 정답 표기(예: '①' 또는 인덱스)
- 해설: 정답에 관한 설명 또는 정오 수정 설명

필수 규칙
1) 자료 제한: 선택지와 해설만 사용하세요. 원 질문 문구가 있더라도 주제 추론에 활용하지 마세요.
2) 주제 추출: 선택지와 해설에서 반복적으로 등장하는 핵심 명사구를 찾고, 가장 포괄적·간결한 1개의 주제를 도출하세요.
3) 서술형 변환: “[주제]에 관해 서술하시오.” 형태로 만들되, 앞에 “다음 키워드를 활용하여”를 붙여 한 문장으로 제시하세요.
4) 선지 분류: 해설을 근거로 틀린 선지 1개와 옳은 선지 3~4개를 식별하세요. 옳은 선지가 3개 미만이면 가능한 만큼만 사용하세요.
   - 정답 표기가 있는 경우, 해설과의 정합성을 우선으로 판단하여 틀린 선지를 확정하세요.
5) 키워드 추출: 틀린 선지 1개와 옳은 선지 3~4개 각각에서 핵심 개념을 1개씩 뽑아 키워드를 구성하세요. 총 5~10개 이내로 하며, 다음 원칙을 지키세요.
   - 정답이 바로 드러나지 않도록 수치·주기·임계치·예외 조건·직접적인 정오 표현을 추상화하십시오.
   - 예: “반기마다 개최” → “개최 주기”, “익일 신고” → “보고 시한”, “상장폐지사유” → “제재 요건”
   - 중복 키워드는 제거하고, 명사구 중심으로 간결히 표현하세요.
6) 출력은 정확히 두 줄만 생성하세요.
   - 1행: 서술형 문제: 다음 키워드를 활용하여 [주제]에 관해 서술하시오.
   - 2행: 키워드: 키워드1, 키워드2, 키워드3, …

검증 체크리스트
- 키워드는 10개 이내인가?
- 키워드에 구체 수치·주기·정답을 직설적으로 드러내는 표현이 없는가?
- 주제 문구가 명확하고 간결한가?

예시 입력 형식
선택지: ['① ...', '② ...', '③ ...', '④ ...

In [14]:
system_prompt = f"""
다음 서술형 질문과 키워드, 원래 질문/선지/정답/해설을 활용하여 모범답안을 만들려고 합니다.
아래 입력 값은 예시로, 아래와 같은 문제들이 들어왔을 때 모범답안을 작성할 수 있는 프롬프트를 작성해주세요.
주어진 정보 이외에 추가적인 정보를 사용하지 않도록 주의해야합니다.
"""

user_prompt = f"""
====== 예시 =====
'서술형 질문': '다음 키워드를 활용하여 독점적 경쟁시장에 관해 서술하시오.\n키워드: 공급자 역할, 장기균형 이윤, 생산량 수준, 상품 차별화, 상품 대체성'
'원래 질문': '독점적 경쟁시장에 관한 설명으로 옳지 않은 것은?',
 '원래 선지': ['① 수많은 공급자가 가격 수용자로 행동한다.',
  '② 장기균형에서 기업들의 경제학적 이윤은 0이다.',
  '③ 장기균형에서는 평균비용곡선의 최저점보다 더 적은 양을 생산한다.',
  '④ 수많은 공급자가 서로 차별화된 상품을 공급하지만, 공급된 상품들의 대체성이 높다.'],
 '원래 정답': '①',
'선지별 해설': '독점적 경쟁시장에서 공급자는 가격 수용자가 아니라 가격 설정자로서 행동한다.\n② 독점적 경쟁시장에 기업들은 정상이윤만 획득하기에 경제학적 이윤은 0이 된다.\n③ 독점적 경쟁시장에서 장기균형조건은 P=SAC=LAC>MR=SMC=LMC 이므로 생산량은 평균비용곡선의 최저점보다 적은 양을 생산한다.\n④ 차별화된 상품들이 재화에 근본적인 차이가 날 정도로 차이가 존재하지 않아 공급된 상품들의 대체성이 높다.'
"""

In [15]:
response = llm.query_openrouter(system_prompt, user_prompt, model_name = 'google/gemini-2.5-pro')
print(response)

## 프롬프트

당신은 주어진 여러 정보를 조합하여 서술형 문제에 대한 '모범답안'을 생성하는 AI입니다.

**[역할]**
- '서술형 질문'의 요구사항을 정확히 파악합니다.
- '원래 질문/선지/정답/해설'에서 서술형 질문에 답변하는 데 필요한 핵심 정보를 추출합니다.
- 추출한 정보와 '키워드'를 논리적으로 엮어 하나의 완성된 글로 재구성합니다.

**[수행 절차]**
1.  **주제 파악:** '서술형 질문'과 '키워드'를 통해 모범답안이 다루어야 할 핵심 주제와 포함해야 할 요소를 확인합니다.
2.  **정보 추출:** '원래 선지'와 '선지별 해설'을 분석하여 각 '키워드'에 해당하는 구체적인 내용을 찾아냅니다.
3.  **논리적 구성:** 추출한 정보들을 활용하여 '서술형 질문'에 대한 답변이 되도록 문장을 자연스럽게 연결하고 문단을 구성합니다. 이때, 모든 '키워드'가 포함되어야 합니다.
4.  **답안 생성:** 위의 과정을 거쳐 최종 '모범답안'을 작성합니다.

**[규칙]**
- **절대 외부 지식을 사용하지 마세요.** 반드시 '원래 질문/선지/정답/해설'에 명시된 정보만을 근거로 답안을 작성해야 합니다.
- 제시된 모든 '키워드'를 반드시 답안에 포함해야 합니다.
- 최종 결과물은 "모범답안:"으로 시작하는 완성된 형태의 글이어야 합니다.

---

**[입력 형식]**
'서술형 질문': {서술형 질문 내용}
'키워드': {키워드 목록}
'원래 질문': {원래 객관식 질문 내용}
'원래 선지': {원래 객관식 선지 목록}
'원래 정답': {원래 객관식 정답}
'선지별 해설': {각 선지에 대한 해설}

**[출력 형식]**
모범답안: {생성된 모범답안}

---

이제 아래 입력 값에 대해 위의 역할, 수행 절차, 규칙에 따라 모범답안을 생성해주세요.


## 평가프롬프트

In [16]:
system_prompt = """당신은 프롬프트 엔지니어입니다. 아래와 같은 서술형 문제와 답변을 받았을 때, 답변이 모든 키워드를 포함하는가를 확인하는 프롬프트를 작성해주세요.
하나라도 키워드가 빠진게 있다면 False, 모두 포함하면 True를 반환합니다.
"""


user_prompt = f"""
서술형 질문: {{서술형질문}}
키워드: {{키워드}}
----
모델 답변(평가대상): {{모델답변}}
"""
response = llm.query_openrouter(system_prompt, user_prompt, model_name = 'openai/gpt-5')
print(response)

다음 지시를 그대로 따르세요. 목적은 모델 답변이 모든 키워드를 포함하는지 여부만 판단하는 것입니다.

역할
- 당신은 키워드 포함 여부만 판단하는 자동 채점기입니다.

출력 형식
- 오직 True 또는 False만 출력합니다. 다른 문자, 공백, 설명, 기호를 절대 추가하지 마세요.

입력
서술형 질문: {서술형질문}
키워드: {키워드}
----
모델 답변(평가대상): {모델답변}

판단 규칙
- 평가는 오직 모델 답변이 키워드를 모두 포함하는지에 한정합니다. 질문의 내용 타당성은 고려하지 않습니다.
- 키워드 목록 파싱
  - 키워드가 JSON 배열 형태면 그대로 사용합니다. 예: ["A", "B"]
  - 그렇지 않으면 쉼표(,), 세미콜론(;), 슬래시(/), 개행(\n)로 분리해 리스트를 만듭니다.
  - 각 키워드는 앞뒤 공백을 제거하고 양쪽 따옴표/괄호/불릿(예: ", ', “ ”, ( ), [ ], { }, -, •) 등 표식은 제거합니다.
  - 빈 문자열이 된 항목은 제외하고, 중복은 제거합니다.
- 정규화(모델 답변과 각 키워드 모두에 적용)
  - 유니코드 정규화(NFKC), 라틴 알파벳은 소문자로 변환합니다.
  - 문장부호와 특수기호(.,!?;:"'()[]{}<>/~`+=_|\)는 무시합니다.
  - 연속 공백은 하나로 축소하고 양끝 공백을 제거합니다.
- 매칭 기준
  - 키워드가 띄어쓰기를 포함하는 구(phrase)인 경우: 정규화된 답변 내에 해당 구가 부분 문자열로 존재하면 포함으로 간주합니다.
  - 키워드가 라틴 알파벳/숫자로만 이루어진 단어인 경우: 단어 경계 기준(영숫자와 비영숫자의 경계)으로 일치해야 포함으로 간주합니다. 예: "ai"는 "aic"에 포함된 것으로 보지 않음.
  - 한글/중국어/일본어 등 공백 구분이 느슨한 언어의 단어는 정규화된 답변에 해당 문자열이 부분적으로 그대로 나타나면 포함으로 간주합니다.
- 동의어, 축약어, 오탈자 치환 등 의미적 유사성은 고려하지 않습니다. 문자열 기준의 포함 여부만 판단